IMPORT DATA

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# import data from the generated dataset 
# generated in DataBuilding.ipnyb
data = pd.read_csv('..\\data\\history\\history.csv')
data.head()

,state,move,pred_prob
0,rnbqkbnr/pppppppp/8/8/8/5N2/PPPPPPPP/RNBQKB1R ...,g8h6,1


In [3]:
data.describe()

,pred_prob
count,1.0
mean,1.0
std,NaN
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


Need to investigate origins and meaning of fen() encoding before examining further aspects of dataset

In [4]:
def bin_matrix_to_decimal(matrix):
    m = matrix.reshape(-1)
    dec = 0
    j = len(m)-1
    for i in range(len(m)-1):
        if m[i]:
            dec += 2**j
        j -= 1
    return float(dec)

In [5]:
def serialize(board):
    import numpy as np
    assert board.is_valid()

    bstate = np.zeros(64, np.uint8)
    for i in range(64):
      pp = board.piece_at(i)
      if pp is not None:
        #print(i, pp.symbol())
        bstate[i] = {"P": 1, "N": 2, "B": 3, "R": 4, "Q": 5, "K": 6, \
                     "p": 9, "n":10, "b":11, "r":12, "q":13, "k": 14}[pp.symbol()]
    if board.has_queenside_castling_rights(chess.WHITE):
      assert bstate[0] == 4
      bstate[0] = 7
    if board.has_kingside_castling_rights(chess.WHITE):
      assert bstate[7] == 4
      bstate[7] = 7
    if board.has_queenside_castling_rights(chess.BLACK):
      assert bstate[56] == 8+4
      bstate[56] = 8+7
    if board.has_kingside_castling_rights(chess.BLACK):
      assert bstate[63] == 8+4
      bstate[63] = 8+7

    if board.ep_square is not None:
      assert bstate[self.board.ep_square] == 0
      bstate[self.board.ep_square] = 8
    bstate = bstate.reshape(8,8)


    state = []
   
    state.append(bin_matrix_to_decimal((bstate>>3)&1))
    state.append(bin_matrix_to_decimal((bstate>>2)&1))
    state.append(bin_matrix_to_decimal((bstate>>1)&1))
    state.append(bin_matrix_to_decimal((bstate>>0)&1))

    state.append(((board.turn*1.0)+50))
    
    return state


In [6]:
import chess

# to change encoded byte strings above into
# normal string format
def serialize2(board):
    val = serialize(chess.Board(str(board)))
    return val

# apply here
data['state'] = data['state'].apply(serialize2)
data = pd.concat([data['state'].apply(pd.Series), data['move']], axis = 1)
data = data.rename(columns = {0:'b1',1:'b2',2:'b3',3:'b4',4:'b5'})

In [7]:
data.head()

,b1,b2,b3,b4,b5,move
0,65534.0,1.102481e+19,1.707765e+19,1.311420e+19,50.0,g8h6


In [8]:
data['b1'] = data['b1'].apply(np.log)
data['b2'] = data['b2'].apply(np.log)
data['b3'] = data['b3'].apply(np.log)
data['b4'] = data['b4'].apply(np.log)

classes = np.random.randint(2, size=len(data.index))
data['class'] = classes
data.head()

,b1,b2,b3,b4,b5,move,class
0,11.090324,43.84668,44.284303,44.020227,50.0,g8h6,1


In [9]:
def listify(string):
    return list(string)

In [10]:
move = data.pop('move')
data = pd.concat([data,move.apply(listify).apply(pd.Series)], axis = 1)
data = data.rename(columns = {0:'m1',1:'m2',2:'m3',3:'m4',4:'m5'})
ism5 = 'm5' in data.columns
if not ism5:
    data['m5'] = 0
data.head(10)                                                              

,b1,b2,b3,b4,b5,class,m1,m2,m3,m4,m5
0,11.090324,43.84668,44.284303,44.020227,50.0,1,g,8,h,6,0


In [11]:
def encode_move(m):
    try:
        val = ord(m)
        return val
    except Exception:
        if not isinstance(m, int):
            raise TypeError('Encoded Move must be of type str or int')
        else:
            return int(m)


data['m1'] = data['m1'].apply(encode_move)
data['m2'] = data['m2'].apply(encode_move)
data['m3'] = data['m3'].apply(encode_move)
data['m4'] = data['m4'].apply(encode_move)
data['m5'] = data['m5'].apply(encode_move)
    
data.head()

,b1,b2,b3,b4,b5,class,m1,m2,m3,m4,m5
0,11.090324,43.84668,44.284303,44.020227,50.0,1,103,56,104,54,0


In [12]:
_class = data.pop('class')
data = pd.concat([data,_class], axis = 1)
data.head()

,b1,b2,b3,b4,b5,m1,m2,m3,m4,m5,class
0,11.090324,43.84668,44.284303,44.020227,50.0,103,56,104,54,0,1


In [13]:
from sklearn import svm

data.pop('class')
y =  np.random.randint(2, size=len(data.index))
s = svm.SVC(kernel = 'rbf', gamma=1/64, probability = True)
s.fit(data, y)
test = [{'b1':11.090324,'b2':11.090324,'b3':11.090324,'b4':11.090324,'b5':11.090324,'m1':11.090324,'m2':11.090324,'m3':11.090324,'m4':11.090324}]
print(s.predict_proba(pd.DataFrame(test)))

ValueError: The number of classes has to be greater than one; got 1 class

In [ ]:
print(s.classes_)